# Multilayer Perceptron - One Hidden Layer

Notation: The superscript $[i]$ represents the $i^{th}$ observation, while $(i)$ represents the $i^{th}$ layer.

Assume a fully connected NN, i.e Multilayer Perceptrons.

Training dataset: $\mathcal{D}_{train}$

Size of data: $N$ 

No. of input features: $d$

No. of outputs: $q$

Input matrix = $\mathbf{x} \in \mathbb{R}^{d \times 1}$

No. of hidden units: $h$

Hidden-layer weights: $\mathbf{W}^{(1)} \in \mathbb{R}^{h \times d}$

Hidden-layer biases: $\mathbf{b}^{(1)} \in \mathbb{R}^{h \times 1}$

Output of hidden units: $\mathbf{h} \in \mathbb{R}^{h \times 1}$

Output-layer weights: $\mathbf{W}^{(2)} \in \mathbb{R}^{q \times h}$

Output-layer biases: $\mathbf{b}^{(2)} \in \mathbb{R}^{q \times 1}$

A nonlinear-activation function: $\sigma$

And the output: $\mathbf{o} \in \mathbb{R}^{q \times 1}$

Thus the mathematical representation of our model is


$$\mathbf{H} = \sigma(\mathbf{W}^{(1)} \mathbf{x} + \mathbf{b}^{(1)})$$

$$\mathbf{o} = \mathbf{W}^{(2)} \mathbf{h} + \mathbf{b}^{(2)}$$

## Concrete Mathematical Representation


$N = 4898$, $d=11$, $h = 22$

`x` $\leftarrow$ $\mathbf{x} \in \mathbb{R}^{11 \times 1}$

`W1` $\leftarrow$ $\mathbf{W}^{(1)} \in \mathbb{R}^{22 \times 11}$

`b1` $\leftarrow$ $\mathbf{b}^{(1)} \in \mathbb{R}^{22 \times 1}$

`H` $\leftarrow$ $\mathbf{h} \in \mathbb{R}^{22 \times 1}$

`W2` $\leftarrow$ $\mathbf{w}^{(2)T} \in \mathbb{R}^{1 \times 22}$

`b2` $\leftarrow$ $b^{(2)} \in \mathbb{R}^{1}$

ReLU activation: $\sigma(x) = \max(x, 0)$

`y_pred` $\leftarrow$ $o \in \mathbb{R}^1$


### Model's Equations

$$\mathbf{h} = \sigma(\mathbf{W}^{(1)} \mathbf{x} + \mathbf{b}^{(1)})$$

$$o = \mathbf{w}^{(2)T} \mathbf{h} + \mathbf{b}^{(2)}$$


### Loss function and Empirical Risk Function

Let loss $L$, be $L = \mathscr{l}(o, y)$

`loss_fn` $\leftarrow$ $l(o, y) = \frac{1}{2} (y - o)^2$

And the empirical risk $J = \frac{1}{n} \sum_{i = 1}^{n} \mathscr{c}(o^{[i]}, y^{[i]})$


### Gradients


#### Empirical Risk

$$\frac{\partial J}{\partial L} = 1$$

#### Loss

$$\frac{\partial L}{\partial o} = -(y - o)$$

#### Hidden to Output Layer

##### Weights $\frac{\partial J}{\partial \mathbf{w}^{(2)}}$

$$
\frac{\partial J}{\partial \mathbf{w}^{(2)}}
=
\frac{\partial J}{\partial L}
\times
\frac{\partial L}{\partial o}
\times
\frac{\partial o}{\partial \mathbf{w}^{(2)}}
$$

$$
\frac{\partial J}{\partial \mathbf{w}^{(2)}} 
= -(y-o)
\times
\frac{\partial (\mathbf{w}^{(2)T} \mathbf{h})}{\partial \mathbf{w}^{(2)}}
$$


$$
\frac{\partial J}{\partial \mathbf{w}^{(2)}} 
= -(y-o) 
\mathbf{h}
$$

##### Biases $\frac{\partial J}{\partial b^{(2)}}$

$$
\frac{\partial J}{\partial \mathbf{b}^{(2)}}
=
\frac{\partial J}{\partial L}
\times
\frac{\partial L}{\partial o}
\times
\frac{\partial o}{\partial \mathbf{b}^{(2)}}
$$

$$\frac{\partial J}{\partial \mathbf{b}^{(2)}} = -(y-o)$$



#### Input to Hidden Layer


##### Weights $\frac{\partial J}{\partial \mathbf{W}^{(1)}}$

Let $\mathbf{z}$ be the intermediate variable before for calculating elementwise activation of $\sigma$, i.e. $\mathbf{z} = \mathbf{W}^{(1)} \mathbf{x} + \mathbf{b}^{(1)}$

$$
\frac{\partial J}{\partial \mathbf{W}^{(1)}}
=
\frac{\partial J}{\partial L}
\times
\frac{\partial L}{\partial o}
\times
\frac{\partial o}{\partial \mathbf{h}}
\times
\frac{\partial \mathbf{h}}{\partial \mathbf{z}}
\times
\frac{\partial (\mathbf{W}^{(1)} \mathbf{x})}{\partial \mathbf{W}^{(1)}}
$$

$$
\frac{\partial J}{\partial \mathbf{W}^{(1)}}
=
\left[
\left[
- (y - o)
\mathbf{w}^{(2)}
\right]
\odot
\sigma^{\prime}(z)
\right]
\mathbf{x}^{T}
$$


##### Biases $\frac{\partial J}{\partial \mathbf{b}^{(1)}}$

$$
\frac{\partial J}{\partial \mathbf{b}^{(1)}}
=
\left[
- (y - o)
\mathbf{w}^{(2)}
\right]
\odot
\sigma^{\prime}(z)
$$

In [ ]:
from collections import Counter

import numpy as np
import pandas as pd
from IPython.display import clear_output, display

from nnfs.activations import Activation
from nnfs.losses import LossFunction
from nnfs.optimizers import Optimizer
from nnfs.utils import Preprocessing

np.set_printoptions(precision=4)

# configuration
TEST_RATIO = 0.1

In [ ]:
class MultilayerPerceptron:
    def __init__(
        self,
        n_features: int,
        hidden_size: int,
        act_fn: Activation,
        loss_fn: LossFunction,
        optimizer: Optimizer,
    ) -> None:
        self.w1, self.b1 = self.init_theta((hidden_size, n_features), (hidden_size, 1))
        self.w2, self.b2 = self.init_theta((1, hidden_size), (1, 1))

        self._activation = act_fn.forward
        self._activation_gradient = act_fn.backward

        self._loss = loss_fn
        self._loss_gradient = loss_fn.backward
        self.accuracy = loss_fn.accuracy

        self._optimizer = optimizer

    def _risk(self, y_trues: np.ndarray, y_preds: np.ndarray) -> float:
        return float(np.mean(self._loss(y_trues, y_preds), axis=1))

    def _theta2_gradient(self, y_true: np.ndarray, y_preds: np.ndarray, h: np.ndarray):
        b2_gradient = self._loss_gradient(y_true, y_preds)
        w2_gradient = (h * np.mean(b2_gradient, axis=-1, keepdims=True)).T
        return w2_gradient, b2_gradient

    def _theta1_gradient(
        self, y_true: np.ndarray, y_preds: np.ndarray, xs: np.ndarray
    ) -> tuple[np.ndarray, np.ndarray]:
        lhs = self._loss_gradient(y_true, y_preds) * self.w2.T
        rhs = self._activation_gradient(self._layer1_forward(xs))
        b1_gradient = np.multiply(lhs, rhs)
        w1_gradient = b1_gradient @ xs.T

        return w1_gradient, b1_gradient

    def _layer1_activation(self, xs: np.ndarray) -> np.ndarray:
        return self._activation(self._layer1_forward(xs))

    def _layer1_forward(self, xs: np.ndarray) -> np.ndarray:
        return (self.w1 @ xs) + self.b1

    def _layer2_forward(self, h: np.ndarray) -> np.ndarray:
        return (self.w2 @ h) + self.b2

    def forward(self, xs: np.ndarray) -> np.ndarray:
        return self._layer2_forward(self._layer1_activation(xs))

    def update_theta(self, xs: np.ndarray, y_trues: np.ndarray, y_preds: np.ndarray):
        h = self._layer1_activation(xs)
        dw1, db1 = self._theta1_gradient(y_trues, y_preds, xs)
        # print(np.abs(dw1))
        dw2, db2 = self._theta2_gradient(y_trues, y_preds, h)

        self.w1 = self._optimizer.update(self.w1, dw1)
        self.b1 = self._optimizer.update(
            self.b1, np.mean(db1, axis=1).reshape(self.b1.shape)
        )

        self.w2 = self._optimizer.update(
            self.w2, np.mean(dw2, axis=0).reshape(self.w2.shape)
        )

        self.b2 = self._optimizer.update(
            self.b2, np.mean(db2, axis=1).reshape(self.b2.shape)
        )

    def train(
        self,
        xs: np.ndarray,
        ys: np.ndarray,
        epochs: int,
        batch_size: int,
    ):
        n = xs.shape[1]

        for i in range(epochs):
            total_epoch_risk = 0.0
            updates = 0

            start = 0
            end = start + batch_size

            while end <= n:
                xb = xs[:, start:end]
                yb = ys[:, start:end]
                ob = self.forward(xb)
                self.update_theta(xb, yb, ob)

                total_epoch_risk += self._risk(yb, ob)
                updates += 1
                start += batch_size
                end += batch_size

            if (i + 1) % 5 == 1:
                clear_output(wait=True)
            print(f"Epoch {i + 1} risk: {np.round(total_epoch_risk / updates, 4)}")

    def evaluate(self, x_test: np.ndarray, y_test: np.ndarray):
        o_test = self.forward(x_test)
        print(f"R^2 Value: {self.accuracy(y_test.T, o_test.T)}")
        print(f"Empirical Risk: {np.round(self._risk(y_test, o_test), 4)}")
        return o_test

    @staticmethod
    def init_theta(
        weights_shape: tuple[int, int], bias_shape: tuple[int, int]
    ) -> tuple[np.ndarray, np.ndarray]:
        w, b = map(
            lambda x: (x - 0.5) / 100,
            map(np.random.random, (weights_shape, bias_shape)),
        )
        return w, b

In [ ]:
np.random.seed(42)
white_wine_csv = "../data/raw/winequality-white.csv"
red_wine_csv = "../data/raw/winequality-red.csv"

white_wine = pd.read_csv(white_wine_csv, delimiter=";")

display(white_wine)
display(white_wine.describe())
white_wine_raw = white_wine.to_numpy()

red_wine = pd.read_csv(red_wine_csv, delimiter=";")
red_wine_raw = red_wine.to_numpy()

wines_raw = np.concatenate((white_wine_raw, red_wine_raw))
# red_wine_raw

wine_mean: np.ndarray = np.mean(wines_raw, axis=0, keepdims=True)
wines_std: np.ndarray = np.std(wines_raw, axis=0, keepdims=True)

wines_raw = (wines_raw - wine_mean) / wines_std

test, train = Preprocessing.train_test_split(wines_raw, TEST_RATIO, shuffle=True)

x_train, y_train = Preprocessing.xy_split(train)
x_test, y_test = Preprocessing.xy_split(test)

x_train = x_train.T
y_train = y_train.T
x_test = x_test.T
y_test = y_test.T

wine_mean, wines_std = wine_mean.flatten(), wines_std.flatten()

In [ ]:
model = MultilayerPerceptron(
    n_features=x_train.shape[0],
    hidden_size=100,
    act_fn=Activation.sigmoid(),
    loss_fn=LossFunction.squared_error(),
    optimizer=Optimizer.adam(learning_rate=1e-3),
)
model.train(x_train, y_train, epochs=100, batch_size=10)

In [ ]:
y_pred = model.evaluate(x_test, y_test)
y_pred_: np.ndarray = (y_pred * wines_std[-1]) + wine_mean[-1]
y_test_: np.ndarray = (y_test * wines_std[-1]) + wine_mean[-1]


print("Distribution of True Values", Counter(np.round(y_test_).tolist()[0]))
print("Distribution of Predictions", Counter(np.round(y_pred_).tolist()[0]))

In [ ]:
y_pred = model.evaluate(x_train, y_train)
y_pred2: np.ndarray = (y_pred * wines_std[-1]) + wine_mean[-1]
y_train_: np.ndarray = (y_train * wines_std[-1]) + wine_mean[-1]


print("Distribution of True Values", Counter(np.round(y_train_.flatten()).tolist()))
print("Distribution of Predictions", Counter(np.round(y_pred2.flatten()).tolist()))

Log transform for normalizing the input dataset (TODO)

For the input patterns, subtract mean, dot product of transpose, || *like cosine similarity?* (TODO)

Remove confusable patterns from the dataset.  (TODO)

**Prediction on the extremes [1, 10] then [1, 2, 9, 10]** (DONE)

Pick input vectors as weight initializations. (TODO)

RBF activation or Sigmoid xform (TODO)

Compute the eigen vectors of the inputs, and use them as weight inits. <- define the basis set, (TODO)

Or even the covariance matrix. (TODO)

Make the features categorical (TODO)

Subset the wine dataset to create a toy problem (DONE)